# Algoritmos de Centralidad - Grado de Centralidad

## Descripcion del Tipo de Algoritmo 



Los algoritmos de centralidad se utilizan para determinar la importancia de distintos nodos en una red.

## Descripcion del Algoritmo

Este algoritmo mide el número de relaciones entrantes y salientes de un nodo, es la medida más simple y directa de las medidas de centralidad Esta corresponde en términos simples al número de enlaces o conexiones que posee un vértice con el resto de los nodos pertenecientes a un grafo. Esta medida fue propuesta por Linton C. Freeman en un documento en 1979.
Es dedicado a realizar la estimación del grado de centralidad mas utilizado para detectar la popularidad o el grado de influencia individual que posee un vértice dentro de un grado, también ayuda a realizar análisis total para detectar un grado mínimo, medio, máximo y la desviación estándar que existe de un grafo.


# Proceso del algoritmo en Neo4j



1.	Construir un grafo con toda la información:

2. Lo siguiente ejecutara el algoritmo y los resultados de los usuarios que tienen mas seguidores en la Red Social
          
3. Lo siguiente ejecutara el algoritmo y los resultados de la transmision, mostrando que usuarios siguen a la mayoria de los otros usuarios.

## Ejemplo Sencillo

# Paso 1

CREATE (alice:User {name: 'Alice'}),
           
           (bridget:User {name: 'Bridget'}),
           (charles:User {name: 'Charles'}),
           (doug:User {name: 'Doug'}),
           (mark:User {name: 'Mark'}),
           (michael:User {name: 'Michael'}),
           (alice)-[:FOLLOWS]->(doug),
           (alice)-[:FOLLOWS]->(bridget),
           (alice)-[:FOLLOWS]->(charles),
           (mark)-[:FOLLOWS]->(doug),
           (mark)-[:FOLLOWS]->(michael),
           (bridget)-[:FOLLOWS]->(doug),
           (charles)-[:FOLLOWS]->(doug),
           (michael)-[:FOLLOWS]->(doug) 
    
            
<img src="DC Nodos.JPG">


## Paso 2

    CALL gds.alpha.degree.stream({
      nodeProjection: 'User',
      relationshipProjection: {
        FOLLOWS: {
          type: 'FOLLOWS',
          orientation: 'REVERSE'
        }
      }
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score AS followers
    ORDER BY followers DESC

<img src="DC Follows.JPG">

# Paso 3

    CALL gds.alpha.degree.stream({
      nodeProjection: 'User',
      relationshipProjection: 'FOLLOWS'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score AS followers
    ORDER BY followers DESC
    
<img src="DC Seguidos.JPG">

# Uso del algoritmo desde Python 

In [17]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nikoap77"),encrypted=False)

persona = []
nfolls = []

def degreeCentrality(tx):
    result = tx.run("CALL gds.alpha.degree.stream({ "
                    "nodeProjection: 'User', "
                    "relationshipProjection: { "
                    "FOLLOWS: { "
                    "type: 'FOLLOWS', "
                    "orientation: 'REVERSE'}}}) "
                    "YIELD nodeId, score "
                    "RETURN gds.util.asNode(nodeId).name AS name, score AS followers "
                    "ORDER BY followers DESC")
    for record in result:
        persona.append(record["name"])
        nfolls.append(record["followers"])

usuarios = []
nfollowing = []

def degreeCentralityFollowing(tx):
    result = tx.run("CALL gds.alpha.degree.stream({ "
                    "nodeProjection: 'User', "
                    "relationshipProjection: 'FOLLOWS' "
                    "}) "
                    "YIELD nodeId, score "
                    "RETURN gds.util.asNode(nodeId).name AS name, score AS followers "
                    "ORDER BY followers DESC")
    for record in result:
        usuarios.append(record["name"])
        nfollowing.append(record["followers"])
        
        
        
        
        

with driver.session() as session:
    session.read_transaction(degreeCentrality)
    print("Resultado numero de Seguidores")
    print(" ")
    print("Persona" ," -> ", "Followers")
    for i in range(len(persona)):
        print(persona[i] ," -> ", nfolls[i])
    print(" ")
        
    session.read_transaction(degreeCentralityFollowing)
    print("Resultado numero de usuarios siguiendo")
    print(" ")
    print("Persona" ," -> ", "Following")
    for i in range(len(usuarios)):
        print(usuarios[i] ," -> ", nfollowing[i])
        
driver.close()




Resultado numero de Seguidores
 
Persona  ->  Followers
Doug  ->  5.0
Bridget  ->  1.0
Charles  ->  1.0
Michael  ->  1.0
Alice  ->  0.0
Mark  ->  0.0
 
Resultado numero de usuarios siguiendo
 
Persona  ->  Following
Alice  ->  3.0
Mark  ->  2.0
Bridget  ->  1.0
Charles  ->  1.0
Michael  ->  1.0
Doug  ->  0.0


# Ejemplificacion  usando datos reales.

In [8]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nikoap77"),encrypted=False)

iglesias = []
nfollowing = []
def degreeCentrality(tx):
    result = tx.run("CALL gds.alpha.degree.stream({ "
                    "nodeProjection: 'Iglesia', "
                    "relationshipProjection: 'CONNECTION' "
                    "}) "
                    "YIELD nodeId, score "
                    "RETURN gds.util.asNode(nodeId).nombre AS name, score AS following "
                    "ORDER BY following DESC")
    for record in result:
        iglesias.append(record["name"])
        nfollowing.append(record["following"])
        
iglesias1 = []
nfollowing1 = []
def degreeCentrality1(tx):
    result = tx.run("CALL gds.alpha.degree.stream({ "
                    "nodeProjection: 'Iglesia', "
                    "relationshipProjection: { "
                    "FOLLOWS: { "
                    "type: 'CONNECTION', "
                    "orientation: 'REVERSE' "
                    "} "
                    "} "
                    "}) " 
                    "YIELD nodeId, score "
                    "RETURN gds.util.asNode(nodeId).nombre AS name, score AS followers "
                    "ORDER BY followers DESC")
    for record in result:
        iglesias1.append(record["name"])
        nfollowing1.append(record["followers"])
        
        
with driver.session() as session:
    session.read_transaction(degreeCentrality)
    print("Resultado numero de rutas salientes")
    print(" ")
    print("Iglesia" ," -> ", "# Rutas Salientes")
    for i in range(len(iglesias)):
        print(iglesias[i] ," -> ", nfollowing[i])
    print(" ")
    
    session.read_transaction(degreeCentrality1)
    print("Resultado numero de rutas entrantes")
    print(" ")
    print("Iglesia" ," -> ", "# Rutas Entrantes")
    for i in range(len(iglesias1)):
        print(iglesias1[i] ," -> ", nfollowing1[i])
    print(" ")
        




Resultado numero de rutas salientes
 
Iglesia  ->  # Rutas Salientes
I. La Merced  ->  4.0
I. Biblica Cristiana  ->  3.0
I. Universal del Reino de Dios  ->  2.0
I. Catolica Señor de la Buena Esperanza  ->  2.0
I. Catolica Santa Gema  ->  2.0
I. Catolica Nuestra Señora de los Angeles   ->  2.0
I. San Alejo  ->  2.0
I. Catolica San Judas Tadeo  ->  2.0
I. La Redonda Urdesa  ->  2.0
I. El Sagrario  ->  2.0
I. Catolica Santo Domingo de Guzman  ->  1.0
I. San Jose  ->  1.0
I. Galilea  ->  0.0
I. Nuestra Señora de Fatima  ->  0.0
 
Resultado numero de rutas entrantes
 
Iglesia  ->  # Rutas Entrantes
I. Nuestra Señora de Fatima  ->  6.0
I. Galilea  ->  3.0
I. Universal del Reino de Dios  ->  2.0
I. Catolica Señor de la Buena Esperanza  ->  2.0
I. Biblica Cristiana  ->  2.0
I. Catolica Santa Gema  ->  2.0
I. Catolica Santo Domingo de Guzman  ->  2.0
I. La Redonda Urdesa  ->  2.0
I. Catolica Nuestra Señora de los Angeles   ->  1.0
I. San Alejo  ->  1.0
I. San Jose  ->  1.0
I. El Sagrario  ->  1

# Resultados y Analisis.

# Mejoras y recomendaciones

Revisar bien la documentacion ya que en la pagina de Neo4j existen errores o algunas librerias se han actualizado.


# Conclusiones y trabajos futuros